<a href="https://colab.research.google.com/github/nazalnihad/AI_for_good/blob/main/table_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
OPENAI_API_KEY=""

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)


def extract_chunk_from_query(user_query):
    # Replace the following line with your actual logic.
    return "One of the important purposes for which a gat is established is to achieve better price realisation for the crop using its collective strength. For this purpose, it is important that the gat gets together to sell its produce. Every farmer must as part of the spardha enter data pertaining to his/her revenue on the Paani Foundation App. The sum total of therevenue of every member of the gat amounts to the total revenue of the gat. The gatmust specify how much of the total revenue of the gat was obtained by collective sale as a gat along with supporting bills and documents. Marks will be given to the gat depending on the percentage of the total revenue of the gat obtained by collective sale as a gat"

# User's query
user_query = "How is gat established to achieved better price?"
metadata = {"filename": "Document1.pdf", "page_number": "42"}

# Extract a chunk of text from the user's query
extracted_chunk = extract_chunk_from_query(user_query)

# convo set
conversation = [
  {"role": "system", "content": f"You are a helpful assistant. A chunk of text is extracted from the user's query and given. The answer must be based on that chunk of text. Be as helpful and precise as possible. also mention about the file name and pagenumber so the user can check on it later {metadata['filename']} , {metadata['page_number']}. If the question does not match the given chunk, say 'I don't have the information to answer that.'"},
  {"role": "user", "content": user_query},
  {"role": "system", "content": f"The assistant will now provide a response based on the following chunk of text given"},
  {"role": "assistant", "content": extracted_chunk}
]

# Make a request to the OpenAI chat API
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=conversation,
    max_tokens=700,
    temperature=0.7
)

# Extract and print the assistant's reply
assistant_reply = completion.choices[0].message.content


In [ ]:
print(assistant_reply)

The gat is established to achieve better price realization for the crop through collective strength. By coming together to sell their produce, the members of the gat can leverage their collective power to negotiate better prices. This can ultimately lead to improved financial outcomes for the farmers. For further details, you can refer to Document1.pdf, page 42.


In [ ]:
data = {"filename": "Document1.pdf", "page_number": "42"}

data["filename"]

'Document1.pdf'

In [ ]:
# answer from gpt-3.5 turbo
"""To achieve better prices, a gat (a group of farmers) can establish collective strength by working together to sell their produce. By collectively selling their crops,
 the gat can negotiate better prices and increase their overall revenue. This can be done by entering data on the Paani Foundation App, where each member of the gat provides
 information about their revenue. The gat must also provide supporting bills and documents to demonstrate the collective sale. By doing so, the gat can earn marks based on
 the percentage of total revenue obtained through collective sales. For more detailed information, please refer to Document1.pdf, page 42."""

# answer from gpt-3.5 turbo 1106
"""The gat is established to achieve better price realization for the crop through collective strength. By coming together to sell their produce,
 the members of the gat can leverage their collective power to negotiate better prices. This can ultimately lead to improved financial outcomes for
  the farmers. For further details, you can refer to Document1.pdf, page 42."""
